In [ ]:
using Random

In [ ]:
include("../rollout.jl")

In [ ]:
include("../testfns.jl")

### Psuedo-code for Rollout Bayesian Optimization
1. Generate low-discrepancy sequence for Quasi-Monte Carlo
2. Gather initial samples/experimental data
3. Construct the ground truth surrogate model
4. Setup hyperparameters for stochastic gradient descent
5. While budget has not been exhausted
<ol>
    <li>
        Construct a batch of samples for stochastic gradient descent. For each sample
        <ol>
            <li>Create a copy of the ground truth surrogate at the sample location and the pairwise perturbed surrogate.</li>
            <li style="color: #f66">Initialize our trajectory struct with the fantasized surrogate and fantisized perturbed surrogate and fantasy start location.</li>
            <li>Perform rollout on the trajectory for $r$ steps $M_0$ times for Quasi-Monte Carlo integration.</li>
            <li>Update values for $\alpha$ and $\nabla\alpha$</li>
        </ol>
    </li>
    <li>Once SGD has converged, update sample location using update rule</li>
    <li>Save location and value at location for each sample in batch.</li>
    <li>Select the best sample from the batch and sample original process at new sample location.</li>
    <li>Update surrogate model with value found at new sample location.</li>
    <li>Repeat until budget is exhausted.</li>
</ol>

### Issues
- Use control variates to see how they affect the rollout acquisition functions

In [ ]:
function measure_gap(sur::RBFsurrogate, fbest)
    gaps = []
    init_mini = sur.y[1] .+ sur.ymean
    maximum_possible_reduction = init_mini - fbest
    
    for i in 1:length(sur.y)
        cur_mini = minimum(sur.y[1:i]) .+ sur.ymean
        gap = (init_mini - cur_mini) / maximum_possible_reduction
        # if init_mini - cur_mini > maximum_possible_reduction
        #     println("Numerator: $(init_mini - cur_mini) -- Denominator: $maximum_possible_reduction")
        #     println("Current Minimum")
        # end
        push!(gaps, gap)
    end
    
    return gaps
end

In [ ]:
HORIZON = 0
MC_SAMPLES = 100
BUDGET = 15
NUM_TRIALS = 10
MAX_SGD_ITERS = 500
BATCH_SIZE = 36

In [ ]:
# Setup toy problem using synthetic test function
testfn = TestGramacyLee()
fbest = testfn.f(first(testfn.xopt))
lbs, ubs = testfn.bounds[:,1], testfn.bounds[:,2]
initial_samples = randsample(NUM_TRIALS, testfn.dim, lbs, ubs)
lds_rns = gen_low_discrepancy_sequence(MC_SAMPLES, testfn.dim, HORIZON+1)
# batch = generate_batch(BATCH_SIZE; lbs=lbs, ubs=ubs);
batch = range(lbs[1], ubs[1], length=BATCH_SIZE)
batch = reshape(batch, testfn.dim, BATCH_SIZE);

In [ ]:
# Setup Gaussian Process statistical model
ℓ, output_variance, σn2 = [1.], 1., 1e-4
ψ = kernel_scale(kernel_matern52, [output_variance, ℓ...]);

In [ ]:
gaps = []
sur = nothing

for trial in 1:NUM_TRIALS
    println("Starting Trial #$trial")
    # Grab initial sample for each trial
    X = reshape(initial_samples[:, trial], testfn.dim, 1)
    sur = fit_surrogate(ψ, X, testfn.f; σn2=σn2)
    domain = filter(x -> !(x in sur.X), lbs[1]:.01:ubs[1])
    
    println("Beginning Bayesian Optimization Main Loop")
    println("-----------------------------------------")
    for budget in 1:BUDGET
        plot()
        plot1DEI(sur; domain=domain)
        println("Iteration #$budget")
        results = []
        
        # Evaluate α(x) for each batch location and perform SGA
        for j in 1:size(batch, 2)
            x0 = batch[:, j]
            # try-catch guard for rollout at points sufficiently close to history.
            # These locations produce a singular exception since the expected improvement
            # at these locations is expected to be zero.
            try
                res = stochastic_gradient_ascent_adam(x0;
                    max_sgd_iters=MAX_SGD_ITERS, lbs=lbs, ubs=ubs, mc_iters=MC_SAMPLES,
                    lds_rns=lds_rns, horizon=HORIZON, sur=sur
                )
                push!(results, res)
            catch e
                # println(e)
                # println("$x0 too close to point in $(sur.X)")
                continue
            end
        end # END for j in batch
        for r in results
            println("Begin: $(r.start) -- End: $(r.finish) -- Func: $(r.final_obj) -- Grad: $(first(r.final_grad)) -- Iters: $(r.iters)")
        end
        
        # Grab the location with best function evaluation
        best_j = findmax(t -> t.final_obj, results)[2]
        best_result = results[best_j]
        vline!(best_result.start, label="start")
        vline!(best_result.finish, label="finish")
        mm = lpad(string(budget), 3, "0")
        savefig("./ei_$(mm).png")
        
        println("Selected Location: $(best_result.finish)")
        
        # Update model at best location
        sur = update_surrogate(sur, best_result.finish, testfn.f)
        res = optimize_hypers_optim(sur, kernel_matern52; σn2=σn2)
        σ, ℓ = Optim.minimizer(res)
        ψ = kernel_scale(kernel_matern52, [σ, ℓ])
        sur = fit_surrogate(ψ, sur.X, recover_y(sur); σn2=σn2)
        # println("Optimal Kernel Hyperparameters: ($σ, $ℓ)")
    end # END for budget iterations
    
    push!(gaps, measure_gap(sur, fbest))
    println("")
end # END for trial iterations

In [ ]:
2

In [ ]:
domain = filter(x -> !(x in sur.X), lbs[1]:.01:ubs[1])
plot1DEI(sur; domain=domain)
scatter!(sur.X', 0sur.y)
# xstart, xend, f, ∇f, iters = results[fbest_j]
# vline!(xstart, label="Batch Location Start")
# vline!(xend, label="Batch Location End")

In [ ]:
plot1D(sur; domain=domain)

In [ ]:
mean_gaps = []

# Iterate across each column
for j in 1:length(gaps[1])
    push!(mean_gaps, 0.)
    
    for i in 1:length(gaps)
        mean_gaps[j] += gaps[i][j]
    end
    
    mean_gaps[j] /= length(gaps)
end

best_gap_ndx = findmax(g -> g[end], gaps)[2]
worse_gap_ndx = findmin(g -> g[end], gaps)[2]
best_gap = gaps[best_gap_ndx]
worst_gap = gaps[worse_gap_ndx]

In [ ]:
plot(mean_gaps, label="Avg. GAP")
hline!([1.])

In [ ]:
gaps